# 35        


In [6]:
from bs4 import BeautifulSoup
from docx import Document
import os

def extract_text_from_divs(soup):
    text = ""
    # Find all <div> tags containing text
    divs = soup.find_all('div')
    for div in divs:
        # Append text from each <div> tag to the result
        text += div.get_text() + "\n"  # Add a newline after each <div> text
    return text.strip()  # Strip any leading/trailing whitespace

def download_css(css_url):
    # Load local CSS file
    with open(css_url, 'r', encoding='utf-8') as f:
        return f.read()

def apply_css_styles(css_code, doc):
    # Apply CSS styles to the Word document (if needed)
    pass  # You can implement this if you have specific requirements

def convert_html_to_docx(html_file, output_docx):
    # Load HTML content from file
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from <div> tags
    text = extract_text_from_divs(soup)

    # Create a new Word document
    doc = Document()

    # Add extracted text to the Word document
    doc.add_paragraph(text)

    # Find and load local CSS file
    css_file = os.path.join(os.path.dirname(html_file), 'css', 'style.css')
    if os.path.exists(css_file):
        css_code = download_css(css_file)
        apply_css_styles(css_code, doc)

    # Save the Word document
    doc.save(output_docx)

# Example usage:
html_file = 'index.html'
output_docx = 'output.docx'
convert_html_to_docx(html_file, output_docx)


# getting images
